In [ ]:
# -*- coding: utf-8 -*-

# Connect_Azure_Blob.ipynb

# Feb 20, 2023

## Imports ##

from azure.storage.blob import BlobServiceClient

import pandas as pd

from io import StringIO

import time

from Tokens import Azure_Blob_secrets  # custom file


start = time.time()

 
## Functions ##

 

def get_conn_str():

    '''

    Connection string to provide Azure account using https and a key extracted from file

    '''

    myAccountName = Azure_Blob_secrets.get('myAccountName')

    myAccountKey = Azure_Blob_secrets.get('myAccountKey')

    connect_str = f'DefaultEndpointsProtocol=https;AccountName={myAccountName};AccountKey={myAccountKey}'

    return connect_str
 

def get_container_client(connect_str,container):

    '''

    Instantiate a container client object for use with blob items

    '''

    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    container_client = blob_service_client.get_container_client(container=container)

    return container_client

 

def try_connection_azure_blob(container_client):

    try:

        assert container_client

        print(container_client)

        print('connected' )

    except:

        print(container_client)

        print("Sorry, connection NOT established")

 

def pandas_from_blob_csv(container_client, file_name):

    '''

    Download specified blob/file into a StringIO stream, then load to pandas dataframe

    '''

    assert ('.csv' in file_name), 'wrong type of file; expected csv extension'

    # Download blob as StorageStreamDownloader object (stored in memory)

    downloaded_blob = container_client.download_blob(file_name)

    df = pd.read_csv(StringIO(downloaded_blob.content_as_text()), low_memory=False)

    return df

 

def pandas_to_blob_csv(df, container_client, file_name):

    output = df.to_csv(index_label="idx", encoding = "utf-8")

   

    blob_client = container_client.get_blob_client(file_name)

    blob_client.upload_blob(output, overwrite=True)

 

def pandas_from_blob_xlsx(container_client, file_name):

    '''

    Downloads .xlsx files

    '''

    assert ('.xlsx' in file_name), 'wrong type of file; expected csv extension'

    downloaded_blob = container_client.download_blob(file_name)

    df = pd.read_excel(downloaded_blob.content_as_text(encoding=None), engine='openpyxl')

    return df

 

## Main Code ##   

 

if __name__=='__main__':

 

    # List the blobs/files in the container

    connect_str = get_conn_str()

    container= 'sfdcsnapshots'

    container_client = get_container_client(connect_str,container)

 

    try_connection_azure_blob(container_client)